In [1]:
from pyflink.common import WatermarkStrategy, Row
from pyflink.common.serialization import Encoder
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FileSink, OutputFileConfig, NumberSequenceSource
from pyflink.datastream.functions import RuntimeContext, MapFunction
from pyflink.datastream.state import ValueStateDescriptor
from pyflink.common.serialization import JsonRowDeserializationSchema
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FlinkKafkaConsumer,FlinkKafkaProducer
from pyflink.common.serialization import SimpleStringSchema, SerializationSchema,JsonRowSerializationSchema,Encoder
from pyflink.common.typeinfo import Types
import json

In [2]:
# wget -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-sql-connector-kafka/1.15.0/flink-sql-connector-kafka-1.15.0.jar 

env = StreamExecutionEnvironment.get_execution_environment()
# the sql connector for kafka is used here as it's a fat jar and could avoid dependency issues
env.add_jars("file:///opt/flink-1.15.0/lib/flink-sql-connector-kafka-1.15.0.jar")


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.0/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [3]:
deserialization_schema = JsonRowDeserializationSchema.builder().type_info(
                             type_info=Types.ROW_NAMED(
                             ["tag","value"], [Types.STRING(), Types.DOUBLE()])).build()

##Data: {"abc": "123", "xyz": "ddd"}

In [4]:
# connect kafka with spark with simple word count example
# run on a terminal after starting kafka

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test-json
#     kafka-console-producer --bootstrap-server localhost:9092 --topic test-json  

# {"tag": "sensor1", "value": 32.3}
# {"tag": "sensor2", "value": 34.3}

# here we implement windowed word count, the word count would reset every minute, we fix  5 minute window

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test-json-sink
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  test-json-sink  --from-beginning  

In [5]:
kafka_consumer = FlinkKafkaConsumer(
    topics='test-json',
    deserialization_schema=deserialization_schema,
    properties={'bootstrap.servers': 'localhost:9092', 'group.id': 'json-group-1'})

ds = env.add_source(kafka_consumer)

In [6]:
ds.print()
# env.execute()

In [ ]:
serialization_schema = JsonRowSerializationSchema.builder().with_type_info(
    type_info=Types.ROW_NAMED(["tag","value"],[Types.STRING(), Types.DOUBLE()])).build()

kafka_producer = FlinkKafkaProducer(
    topic='test-json-sink',
    serialization_schema=serialization_schema,
    producer_config={'bootstrap.servers': 'localhost:9092'})

ds.add_sink(kafka_producer)

env.execute('state_access_demo')

2> +I[sensor2,34.3]
